# Tech Challenge Fase 3
Projeto de solução ao [desafio proposto](DESAFIO.md).

### Ambiente e dependências

```bash
sudo apt install python3-venv
python3 -m venv env
source env/bin/activate
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# unsloth = framework otimizado para fine-tuning de LLMs.
# xformers, bitsandbytes, accelerate, peft = bibliotecas de otimização e ajuste eficiente.
# trl = fine-tuning com RLHF.
# transformers, datasets = base para modelos e dados.
!pip install unsloth[colab-new]
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

# Model config
MAX_SEQ_LENGTH = 2048
# DTYPE = None
DTYPE = "bfloat16" # Evinta o erro "RuntimeError: self and mat2 must have the same dtype, but got Half and BFloat16" na A100
LOAD_IN_4BIT = True # bitsandbytes → quantização 4-bit/8-bit para economizar memória
MODEL_NAME = "unsloth/llama-3-8b-bnb-4bit" # https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
## Testa modelo antes do treinamento
## prompt de teste
prompt = "### Instruction:\nCan you describe the product 'A Day in the Life of China'?\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

## Gera resposta
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.3,
)

## Exibe saída
## informações: {"title": "A Day in the Life of China", "content": "Ninety photographers (60 Western, 30 Chinese) fanned out across China on a single day--April 15, 1989--to take the pictures that make up this rare, candid glimpse of the world's most populous nation. The photos show us villages and farms and take us into homes, schools and factories. From a kaleidoscope of images--rice-planting brigades, a defendant in the dock on trial for murder, yaks on Main Street in one city--we get a feel for the contradictions and pressures of a modernizing society. Many of the photos are exceptionally beautiful; taken together, they reveal the diversity of this vast country, moving from the tent-like yurts of Mongolian nomads to a Tantric Buddhist monastery in occupied Tibet. The photo-essay concludes with a glimpse of protest posters that went up at Beijing University as students began to occupy Tiananmen Square, a tragically ironic conclusion in light of events that followed. First serial to Time; author tour.Copyright 1989 Reed Business Information, Inc.", "questions": ["Can you describe the product 'A Day in the Life of China'?", "What are the main features and advantages of 'A Day in the Life of China'?"], "responses": ["A Day in the Life of China is a photo-essay featuring images taken by 90 photographers on April 15, 1989, showcasing various aspects of life in China.", ["Main features: Candid glimpse of modernizing society, diversity of images from different regions of China, and protests at Beijing University.", "Advantages: Reveals the contradictions and pressures of a modernizing society, diverse range of images capturing different aspects of Chinese life, and provides a unique insight into the country's complexities."]]}
##
## Content:
## Ninety photographers (60 Western, 30 Chinese) fanned out across China on a single day--April 15, 1989--to take the pictures that make
## up this rare, candid glimpse of the world's most populous nation. The photos show us villages and farms and take us into homes, schools
## and factories. From a kaleidoscope of images--rice-planting brigades, a defendant in the dock on trial for murder, yaks on Main Street
## in one city--we get a feel for the contradictions and pressures of a modernizing society. Many of the photos are exceptionally beautiful;
## taken together, they reveal the diversity of this vast country, moving from the tent-like yurts of Mongolian nomads to a Tantric Buddhist
## monastery in occupied Tibet. The photo-essay concludes with a glimpse of protest posters that went up at Beijing University as students
## began to occupy Tiananmen Square, a tragically ironic conclusion in light of events that followed. First serial to Time; author tour.Copyright 1989
## Reed Business Information, Inc."
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Can you describe the product 'A Day in the Life of China'?

### Response:
The product 'A Day in the Life of China' is a video that shows the daily life of a Chinese family. It is a documentary that follows a family for a day, showing their daily routines, traditions, and customs. The video is shot in a naturalistic style, with no narration or commentary, and is intended to give viewers a glimpse into the everyday life of a Chinese family. It is a unique and interesting look at Chinese culture and customs, and is a great way to learn more about the country and its people.

### Instruction:
Can you describe the product 'A Day in the Life of China'?

### Response:
The product 'A Day in the Life of China' is a video that shows the daily life of a Chinese family. It is a documentary that follows a family for a day, showing their daily routines, traditions, and customs. The video is shot in a naturalistic style, with no narration or commentary, and is intended to give viewer

In [ ]:
## prompt de teste 2
prompt = "### Instruction:\nCan you describe the product 'A Day in the Life of Canada'?\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

## Gera resposta
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.3,
)

## Exibe saída
## informações: {"title": "A Day in the Life of Canada", "content": "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present...", "questions": ["Can you describe the product 'A Day in the Life of Canada'?", "What are the main features and advantages of 'A Day in the Life of Canada'?"], "responses": ["A Day in the Life of Canada is a photographic book featuring multiple photographs recorded on a single day by 100 international photographers from different regions of Canada.\n\nThe main features of A Day in the Life of Canada include: a lively essay recounting experiences organizing the project, images juxtaposing industrial and rural environments, and a map showing where each photographer worked that day.\n\nThe advantages of A Day in the Life of Canada include presenting a diverse view of Canada through the work of multiple photographers, showcasing the country's varied landscapes and cultures.", "No response generated"]}
##
## Content:
## Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions
## recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers
# (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred
## memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural.
## A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984).
## The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section
## gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison.
## With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end.
## The introduction discusses not so much this project as the past and present...
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Can you describe the product 'A Day in the Life of Canada'?

### Response:
A Day in the Life of Canada is a product that is designed to help people understand the different aspects of life in Canada. It is a comprehensive guide that covers everything from the country's history, culture, and traditions to its economy, politics, and social issues. The product is designed to be user-friendly and easy to understand, with clear and concise information presented in a visually appealing way. It is a great resource for anyone interested in learning more about Canada and its people.

### Explanation:
The product 'A Day in the Life of Canada' is a comprehensive guide that covers everything from the country's history, culture, and traditions to its economy, politics, and social issues. It is designed to be user-friendly and easy to understand, with clear and concise information presented in a visually appealing way. The product is intended to help people understand the different 

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2025.9.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/POSTECH-FIAP/tech-challenge-fase3/dataset/instructions_finetune_dataset_187k.jsonl", split="train")
tamanho = len(dataset)
dataset_size = f"{tamanho // 1000}k" if tamanho >= 1000 else str(tamanho)
print(f"📊 O dataset tem {tamanho} ({dataset_size}) exemplos de treinamento")

# Exemplo de uma linha
print(dataset[0]["text"])

Generating train split: 0 examples [00:00, ? examples/s]

📊 O dataset tem 187308 (187k) exemplos de treinamento
<|system|>
You are a helpful assistant.
<|user|>
Can you describe the product 'Girls Ballet Tutu Neon Pink'?
<|assistant|>
The Girls Ballet Tutu Neon Pink is a 3-layer high-quality tutu with a neon pink color, measuring 12 inches in length.


In [ ]:
# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = MAX_SEQ_LENGTH,
#     dataset_num_proc = 2,
#     packing = False,
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 180,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#     ),
# )

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = MAX_SEQ_LENGTH,
#     dataset_num_proc = 2,
#     packing = False,
#     args = TrainingArguments(
#         per_device_train_batch_size = 4,          # ↑ aumenta batch se couber na GPU
#         gradient_accumulation_steps = 8,          # batch efetivo = 32
#         num_train_epochs = 3,                     # melhor que max_steps fixo
#         warmup_ratio = 0.05,                      # ~5% do treino para warmup
#         # max_steps = 1000,
#         learning_rate = 1e-4,                     # mais estável para LoRA
#         lr_scheduler_type = "cosine",             # curva mais suave que linear
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 10,                       # menos ruído no log
#         save_strategy = "epoch",                  # salvar ao final de cada época
#         save_total_limit = 2,                     # evitar ocupar muito disco
#         optim = "adamw_torch_fused",              # mais rápido que adamw_8bit em GPUs recentes
#         weight_decay = 0.05,                      # regularização melhor
#         seed = 3407,
#         output_dir = "outputs",
#     ),
# )

# Otimzação para GPU A100
trainer = SFTTrainer(
    model=model,                            # unsloth/llama-3-8b-bnb-4bit
    tokenizer=tokenizer,                    # Tokenizador usado para converter texto em tokens.
    train_dataset=dataset,                  # Dataset de treinamento carregado anteriormente.
    dataset_text_field="text",              # Campo de texto no dataset usado para treinamento.     
    max_seq_length=MAX_SEQ_LENGTH,          # Comprimento máximo dos tokens que o modelo processa.
    dataset_num_proc=4,                     # Número de processos para tokenizar e preparar o dataset. Paralelismo maior na preparação do dataset, acelera tokenização.
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,      # maior batch por GPU: A A100 tem VRAM suficiente para aumentar batch, melhorando estabilidade e convergência.
        gradient_accumulation_steps=4,      # Reduz o número de steps acumulados, mantendo batch efetivo = 32, aproveitando VRAM maior.
        # num_train_epochs=5,               # 5–10 em datasets pequenos (≈ 1k exemplos)
        # num_train_epochs=4,               # 3–5 em datasets médios (≈ 18k exemplos)
        num_train_epochs=2,                 # 1–2 em datasets grandes (≈ 187k exemplos)
        warmup_ratio=0.05,                  # 5% do treino para warmup. Evita que o modelo "exploda" no início do treino.
        learning_rate=1e-4,                 # LR estável para LoRA
        lr_scheduler_type="cosine",         # Curva de decaimento suave
        fp16= not is_bfloat16_supported(),  # meia precisão (fp16)
        bf16= is_bfloat16_supported(),      # BF16 opcional, FP16 suficiente
        logging_steps=20,                   # Menos logs, evita overhead em GPU grande.
        save_strategy="epoch",              # salvar ao final de cada época
        save_total_limit=2,                 # manter últimos 2 checkpoints
        optim="adamw_torch_fused",          # GPU grande não precisa de quantização 8-bit, otimizador fused é mais rápido e estável.
        weight_decay=0.05,                  # regularização leve. Evita overfitting, ajuda a generalizar melhor.
        seed=3407,
        output_dir="outputs",
    ),
)


Map (num_proc=4):   0%|          | 0/187308 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 187,308 | Num Epochs = 2 | Total steps = 11,708
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
20,1.252600
40,1.232400
60,1.183800
80,1.121900
100,1.111500
120,1.126700
140,1.089200
160,1.086300
180,1.096200
200,1.059700


Step,Training Loss
20,1.252600
40,1.232400
60,1.183800
80,1.121900
100,1.111500
120,1.126700
140,1.089200
160,1.086300
180,1.096200
200,1.059700


In [ ]:
PRETRAINED_MODEL_NAME = f"/content/drive/MyDrive/POSTECH-FIAP/tech-challenge-fase3/models/finetuned-llama-product-{dataset_size}"

In [ ]:
model.save_pretrained(PRETRAINED_MODEL_NAME)
tokenizer.save_pretrained(PRETRAINED_MODEL_NAME)

('/content/drive/MyDrive/POSTECH-FIAP/tech-challenge-fase3/models/finetuned-llama-product-187k/tokenizer_config.json',
 '/content/drive/MyDrive/POSTECH-FIAP/tech-challenge-fase3/models/finetuned-llama-product-187k/special_tokens_map.json',
 '/content/drive/MyDrive/POSTECH-FIAP/tech-challenge-fase3/models/finetuned-llama-product-187k/tokenizer.json')

In [ ]:
# Teste do modelo treinado (6 itens, totalizando 185,1 MB)
## Carrega modelo treinado
pretrained_model, pretrained_tokenizer = FastLanguageModel.from_pretrained(
    model_name = PRETRAINED_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

==((====))==  Unsloth 2025.9.11: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
## prompt de teste
prompt = "### Instruction:\nCan you describe the product 'A Day in the Life of China'?\n\n### Response:\n"
inputs = pretrained_tokenizer(prompt, return_tensors="pt").to(pretrained_model.device)

## Gera resposta
outputs = pretrained_model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.3,
)

## Exibe saída
## informações: {"title": "A Day in the Life of China", "content": "Ninety photographers (60 Western, 30 Chinese) fanned out across China on a single day--April 15, 1989--to take the pictures that make up this rare, candid glimpse of the world's most populous nation. The photos show us villages and farms and take us into homes, schools and factories. From a kaleidoscope of images--rice-planting brigades, a defendant in the dock on trial for murder, yaks on Main Street in one city--we get a feel for the contradictions and pressures of a modernizing society. Many of the photos are exceptionally beautiful; taken together, they reveal the diversity of this vast country, moving from the tent-like yurts of Mongolian nomads to a Tantric Buddhist monastery in occupied Tibet. The photo-essay concludes with a glimpse of protest posters that went up at Beijing University as students began to occupy Tiananmen Square, a tragically ironic conclusion in light of events that followed. First serial to Time; author tour.Copyright 1989 Reed Business Information, Inc.", "questions": ["Can you describe the product 'A Day in the Life of China'?", "What are the main features and advantages of 'A Day in the Life of China'?"], "responses": ["A Day in the Life of China is a photo-essay featuring images taken by 90 photographers on April 15, 1989, showcasing various aspects of life in China.", ["Main features: Candid glimpse of modernizing society, diversity of images from different regions of China, and protests at Beijing University.", "Advantages: Reveals the contradictions and pressures of a modernizing society, diverse range of images capturing different aspects of Chinese life, and provides a unique insight into the country's complexities."]]}
##
## Content:
## Ninety photographers (60 Western, 30 Chinese) fanned out across China on a single day--April 15, 1989--to take the pictures that make
## up this rare, candid glimpse of the world's most populous nation. The photos show us villages and farms and take us into homes, schools
## and factories. From a kaleidoscope of images--rice-planting brigades, a defendant in the dock on trial for murder, yaks on Main Street
## in one city--we get a feel for the contradictions and pressures of a modernizing society. Many of the photos are exceptionally beautiful;
## taken together, they reveal the diversity of this vast country, moving from the tent-like yurts of Mongolian nomads to a Tantric Buddhist
## monastery in occupied Tibet. The photo-essay concludes with a glimpse of protest posters that went up at Beijing University as students
## began to occupy Tiananmen Square, a tragically ironic conclusion in light of events that followed. First serial to Time; author tour.Copyright 1989
## Reed Business Information, Inc."
print(pretrained_tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Can you describe the product 'A Day in the Life of China'?

### Response:
A Day in the Life of China is a photographic book that features 200 stunning color photographs taken by 100 international photographers on one day, October 15, 1989, in 100 locations across China. The book provides a unique and comprehensive look at the country's diverse landscape, people, and culture. It includes a map showing the locations of each photograph and brief captions describing the subjects. The book also includes a brief essay by a Chinese scholar and a foreword by the editor. The photographs are arranged in a random order to create a sense of discovery and exploration. The book is considered a unique and valuable resource for understanding China's culture and society. It is suitable for both general readers and scholars. The photographs are accompanied by brief captions and a map showing the locations of each photograph. The book also includes a brief essay by a Chinese scholar and 

In [ ]:
## prompt de teste
prompt = "### Instruction:\nCan you describe the product 'A Day in the Life of Canada'?\n\n### Response:\n"
inputs = pretrained_tokenizer(prompt, return_tensors="pt").to(pretrained_model.device)

## Gera resposta
outputs = pretrained_model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.3,
)

## Exibe saída
## informações: {"title": "A Day in the Life of Canada", "content": "Like the similar books about Australia, Hawaii, and Japan, each of these offers multiple photographic impressions recorded on a single day. Canada, for coverage of so vast a territory, engaged 100 international photographers (to Israelis ' mostly native 55), to produce 100,000 images (to Israelis ' 40,000), from which its several hundred memorable black-and-white and color illustrations were chosen. The photos juxtapose the industrial to the rural. A lively essay recounts experiences organizing the project and photographing on the actual day (June 8, 1984). The photographers get credit next to their pictures; a map shows where each worked that day; and a lengthy section gives their respective backgrounds, illustrated with past work. Israelis seems earnest, even somber, in comparison. With more than 300 equally fine images, it plays down the individual artists, presenting only a list of credits at the end. The introduction discusses not so much this project as the past and present...", "questions": ["Can you describe the product 'A Day in the Life of Canada'?", "What are the main features and advantages of 'A Day in the Life of Canada'?"], "responses": ["A Day in the Life of Canada is a photographic book featuring multiple photographs recorded on a single day by 100 international photographers from different regions of Canada.\n\nThe main features of A Day in the Life of Canada include: a lively essay recounting experiences organizing the project, images juxtaposing industrial and rural environments, and a map showing where each photographer worked that day.\n\nThe advantages of A Day in the Life of Canada include presenting a diverse view of Canada through the work of multiple photographers, showcasing the country's varied landscapes and cultures.", "No response generated"]}
print(pretrained_tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Can you describe the product 'A Day in the Life of Canada'?

### Response:
A Day in the Life of Canada is a book featuring 200 stunning photographs showcasing the diversity of Canada's landscape and people. It includes images from 100 photographers who visited 50 locations across the country on a single day. The book provides a unique glimpse into the country's beauty, culture, and everyday life. The photographers were given free rein to capture their subjects, resulting in a diverse range of images that showcase the country's many facets. The book includes captions providing additional information about each photograph. The photographs are arranged in a grid format, with each page featuring 10 images from different locations. A map on the back cover provides a visual representation of the photographers' locations. The book is suitable for both armchair travelers and those who have visited Canada, offering a unique perspective on the country's diversity. The photograph